# Single Player Feature Research Notebook

Research and build features for a specific player using backtest workflow.

**Workflow (mirrors backtest process):**
1. Load DFS salaries for target date (the "slate")
2. Select a player from the slate
3. Load historical data for that player (excluding target date)
4. Build features using only historical data
5. Analyze and visualize features

This ensures feature engineering matches production backtest logic.

## Setup

In [8]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

repo_root = Path.cwd().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from src.data.storage.sqlite_storage import SQLiteStorage
from src.data.loaders.historical_loader import HistoricalDataLoader
from src.utils.fantasy_points import calculate_dk_fantasy_points

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print('Setup complete')

Setup complete


## Configuration

Set the target date and number of seasons to load.

**NUM_SEASONS**: 
- `1` = Current season only (from Oct 1 of season year)
- `2` = Current + previous season (default, ~16 months of data)
- `3` = Current + 2 previous seasons, etc.

In [9]:
TARGET_DATE = '20250210'
NUM_SEASONS = 2  # Current season + previous season

DB_PATH = repo_root / 'nba_dfs.db'

storage = SQLiteStorage(str(DB_PATH))
loader = HistoricalDataLoader(storage)

print(f'Target Slate Date: {TARGET_DATE}')
print(f'Seasons to load: {NUM_SEASONS} (current + previous)')
print(f'Database: {DB_PATH}')

Target Slate Date: 20250210
Seasons to load: 2 (current + previous)
Database: c:\Users\antho\OneDrive\Documents\Repositories\delapan-fantasy\nba_dfs.db


## Step 1: Load Slate Data (DFS Salaries)

This is the first step in the backtest process - load salaries for the target date.

In [10]:
print(f'Loading slate data for {TARGET_DATE}...')

slate_data = loader.load_slate_data(TARGET_DATE)

salaries_df = slate_data.get('dfs_salaries', pd.DataFrame())

if salaries_df.empty:
    print(f'ERROR: No salary data found for {TARGET_DATE}')
    print('\nTry a different date. Available dates:')
    
    available_dates = loader.load_slate_dates('20250101', '20250331')
    print(available_dates[:20])
else:
    print(f'\nSlate loaded successfully!')
    print(f'Players on slate: {len(salaries_df)}')
    
    if 'longName' in salaries_df.columns and 'playerName' not in salaries_df.columns:
        salaries_df['playerName'] = salaries_df['longName']
    
    print(f'\nSlate summary:')
    if 'salary' in salaries_df.columns:
        salaries_df['salary'] = pd.to_numeric(salaries_df['salary'], errors='coerce')
        print(f'  Average salary: ${salaries_df["salary"].mean():,.0f}')
        print(f'  Min salary: ${salaries_df["salary"].min():,.0f}')
        print(f'  Max salary: ${salaries_df["salary"].max():,.0f}')
    
    if 'pos' in salaries_df.columns:
        print(f'\nPlayers by position:')
        print(salaries_df['pos'].value_counts())
    
    print(f'\nTop 20 highest salaries on slate:')
    display_cols = ['playerName', 'team', 'pos', 'salary']
    display_cols = [col for col in display_cols if col in salaries_df.columns]
    
    if display_cols:
        top_salaries = salaries_df.nlargest(20, 'salary')[display_cols]
        display(top_salaries)

Loading slate data for 20250210...

Slate loaded successfully!
Players on slate: 342

Slate summary:
  Average salary: $4,623
  Min salary: $3,000
  Max salary: $12,700

Players by position:
pos
SF    84
SG    83
PG    60
PF    59
C     56
Name: count, dtype: int64

Top 20 highest salaries on slate:


,playerName,team,pos,salary
0,Nikola Jokic,DEN,C,12700
1,Giannis Antetokounmpo,MIL,PF,11700
2,Luka Doncic,LAL,PG,11000
3,Victor Wembanyama,SA,C,11000
4,Shai Gilgeous-Alexander,OKC,SG,10800
5,LeBron James,LAL,SF,10700
6,Anthony Davis,DAL,C,10500
7,Jayson Tatum,BOS,PF,10000
8,LaMelo Ball,CHA,PG,9900
9,Trae Young,ATL,PG,9800


## Step 2: Select Player from Slate

Choose a player to analyze. You can select by:
- Index from the table above
- Player name (partial match)

In [11]:
PLAYER_SELECTION = 'Nikola Jokic'

if not salaries_df.empty:
    if isinstance(PLAYER_SELECTION, int):
        selected_player = salaries_df.iloc[PLAYER_SELECTION]
    else:
        matches = salaries_df[
            salaries_df['playerName'].str.contains(PLAYER_SELECTION, case=False, na=False)
        ]
        
        if matches.empty:
            print(f'ERROR: No player matching "{PLAYER_SELECTION}" found on slate')
            print('\nAvailable players:')
            print(salaries_df['playerName'].tolist())
            selected_player = None
        elif len(matches) > 1:
            print(f'Multiple players match "{PLAYER_SELECTION}":')
            display(matches[['playerName', 'team', 'pos', 'salary']])
            print('\nUsing first match. Be more specific if needed.')
            selected_player = matches.iloc[0]
        else:
            selected_player = matches.iloc[0]
    
    if selected_player is not None:
        player_id = selected_player['playerID']
        player_name = selected_player['playerName']
        player_team = selected_player.get('team', 'N/A')
        player_pos = selected_player.get('pos', 'N/A')
        player_salary = selected_player.get('salary', 0)
        
        print('='*60)
        print('SELECTED PLAYER')
        print('='*60)
        print(f'Name: {player_name}')
        print(f'ID: {player_id}')
        print(f'Team: {player_team}')
        print(f'Position: {player_pos}')
        print(f'Salary: ${player_salary:,.0f}')
        print(f'Target Date: {TARGET_DATE}')
        print('='*60)

SELECTED PLAYER
Name: Nikola Jokic
ID: 28908111729
Team: DEN
Position: C
Salary: $12,700
Target Date: 20250210


## Step 3: Load Historical Data for Selected Player

Load player game logs BEFORE the target date (mimics backtest logic).

Loads data for current NBA season + previous season(s) based on NUM_SEASONS parameter.

In [ ]:
if selected_player is not None:
    print(f'Loading historical data for {player_name}...')
    print(f'Loading {NUM_SEASONS} seasons of data (current + previous)')
    
    all_historical_data = loader.load_historical_player_logs(
        end_date=TARGET_DATE,
        num_seasons=NUM_SEASONS
    )
    
    print(f'\nTotal games loaded (all players): {len(all_historical_data)}')
    
    player_data = all_historical_data[
        all_historical_data['playerID'] == player_id
    ].copy()
    
    if player_data.empty:
        print(f'\nERROR: No historical data found for {player_name}')
        print(f'Player may be new or have no games in database')
    else:
        player_data['gameDate'] = pd.to_datetime(player_data['gameDate'], format='%Y%m%d', errors='coerce')
        player_data = player_data.sort_values('gameDate').reset_index(drop=True)
        
        if 'fpts' not in player_data.columns:
            player_data['fpts'] = player_data.apply(calculate_dk_fantasy_points, axis=1)
        
        numeric_cols = ['pts', 'reb', 'ast', 'stl', 'blk', 'TOV', 'mins', 'fpts']
        for col in numeric_cols:
            player_data[col] = pd.to_numeric(player_data[col], errors='coerce')

        if 'season' not in player_data.columns:
            def get_season(date):
                year = date.year
                if date.month >= 7:
                    return f"{year}-{str(year+1)[-2:]}"
                else:
                    return f"{year-1}-{str(year)[-2:]}"
            player_data['season'] = player_data['gameDate'].apply(get_season)

        player_data = player_data.sort_values(['season', 'gameDate']).reset_index(drop=True)
        
        for col in numeric_cols:
            expanding_mean = player_data.groupby('season')[col].expanding().mean()
            expanding_std = player_data.groupby('season')[col].expanding().std()
            
            player_data[f'{col}_season_avg'] = expanding_mean.reset_index(level=0, drop=True)
            player_data[f'{col}_season_std'] = expanding_std.reset_index(level=0, drop=True)

        print(f'\nHistorical games for {player_name}: {len(player_data)}')
        print(f'Date range: {player_data["gameDate"].min().strftime("%Y-%m-%d")} to {player_data["gameDate"].max().strftime("%Y-%m-%d")}')
        
        current_season_start = loader.get_season_start_date(TARGET_DATE)
        current_season_games = player_data[player_data['gameDate'] >= current_season_start]
        print(f'\nCurrent season (from {current_season_start}): {len(current_season_games)} games')
        print(f'Previous season: {len(player_data) - len(current_season_games)} games')
        
        target_date_dt = pd.to_datetime(TARGET_DATE, format='%Y%m%d')
        if player_data['gameDate'].max() >= target_date_dt:
            print('\n*** WARNING: LOOKAHEAD BIAS DETECTED ***')
            print('Historical data includes target date or later!')
        else:
            days_before = (target_date_dt - player_data['gameDate'].max()).days
            print(f'\nTemporal validation: PASSED')
            print(f'Most recent game is {days_before} days before target date')
        
        print(f'\nLast 5 games (most recent data available for features):')
        display(player_data.tail())

Loading historical data for Nikola Jokic...
Loading 2 seasons of data (current + previous)

Total games loaded (all players): 44208

Historical games for Nikola Jokic: 131
Date range: 2023-10-24 to 2025-02-08

Current season (from 20241001): 47 games
Previous season: 84 games

Temporal validation: PASSED
Most recent game is 2 days before target date

Last 5 games (most recent data available for features):


,playerID,longName,team,teamAbv,teamID,gameID,gameDate,pos,mins,pts,reb,ast,stl,blk,TOV,PF,fga,fgm,fgp,fta,ftm,ftp,tptfga,tptfgm,tptfgp,OffReb,DefReb,fantasyPoints,fantasyPts,plusMinus,usage,tech,created_at,fpts,season,pts_season_avg,pts_season_std,reb_season_avg,reb_season_std,ast_season_avg,ast_season_std,stl_season_avg,stl_season_std,blk_season_avg,blk_season_std,TOV_season_avg,TOV_season_std,mins_season_avg,mins_season_std,fpts_season_avg,fpts_season_std
126,28908111729,Nikola Jokic,DEN,DEN,8,20250201_DEN@CHA,2025-02-01,,37,28,13,17,4,1,4,2,17,9,52.9,9,8,88.9,7,2,28.6,1,12,80.75,,-2,30.04,0,2025-10-06 12:33:01,80.75,2024-25,29.642857,9.336060,12.833333,4.751979,10.142857,3.719241,1.809524,1.234427,0.642857,0.790845,3.166667,1.793167,36.404762,4.030733,66.220238,14.978766
127,28908111729,Nikola Jokic,DEN,DEN,8,20250203_NO@DEN,2025-02-03,,36,27,14,10,1,2,1,2,13,9,69.2,10,8,80.0,3,1,33.3,1,13,67.5,,+11,22.42,0,2025-10-06 12:33:01,68.00,2024-25,29.604651,9.227649,12.837209,4.695135,10.302326,3.820589,1.860465,1.264561,0.651163,0.783269,3.186047,1.776243,36.418605,3.983493,66.558140,14.964326
128,28908111729,Nikola Jokic,DEN,DEN,8,20250205_NO@DEN,2025-02-05,,36,38,8,10,1,1,1,3,28,15,53.6,3,3,100.0,13,5,38.5,2,6,68.0,,+24,34.97,0,2025-10-06 12:33:01,68.00,2024-25,29.545455,9.128169,12.863636,4.643530,10.295455,3.776177,1.840909,1.256484,0.681818,0.800370,3.136364,1.786134,36.409091,3.937407,66.590909,14.790896
129,28908111729,Nikola Jokic,DEN,DEN,8,20250206_ORL@DEN,2025-02-06,,31,28,10,12,2,0,3,2,16,11,68.8,6,4,66.7,4,2,50.0,0,10,61.5,,+22,29.85,0,2025-10-06 12:33:02,64.00,2024-25,29.733333,9.111431,12.755556,4.647363,10.288889,3.733279,1.822222,1.248433,0.688889,0.792643,3.088889,1.794211,36.400000,3.892884,66.622222,14.623361
130,28908111729,Nikola Jokic,DEN,DEN,8,20250208_DEN@PHO,2025-02-08,,29,26,11,9,0,0,2,4,13,11,84.6,3,3,100.0,2,1,50.0,4,7,51.25,,+19,24.67,0,2025-10-06 12:33:02,53.75,2024-25,29.695652,9.013248,12.695652,4.613360,10.326087,3.700176,1.826087,1.234762,0.673913,0.790340,3.086957,1.774211,36.282609,3.930864,66.565217,14.465134


## Step 4: Build Features (Exactly as Backtest Does)

Calculate features using ONLY historical data (prior games).

In [68]:
if selected_player is not None and not player_data.empty:
    print('Building features from historical data...')
    print('Restricting to only current and last season data...')
    print('Using .shift(1) to ensure no current-game leakage')

    # Drop columns if they exist, otherwise ignore errors
    features_df = player_data.drop(['fantasyPoints', 'fantasyPts', 'created_at'], axis=1, errors='ignore')
    
    # Make target: always use fpts.shift(-1)
    features_df['target'] = features_df['fpts'].shift(-1)
    
    # Confirm target column is present immediately
    print(f"Columns at target creation: {features_df.columns.tolist()}")

    # Add a "season" column if it doesn't already exist
    def get_season(date):
        year = date.year
        if date.month >= 7:
            return year + 1
        else:
            return year

    features_df['season'] = features_df['gameDate'].apply(get_season)
    features_df = features_df[features_df.season.isin([features_df.season.max(), features_df.season.max()-1])]

    # Restrict to only current and last season's data
    if 'season' in features_df.columns:
        unique_seasons = sorted(features_df['season'].unique())
        # Find current season (latest) and last season (second latest if present)
        if len(unique_seasons) > 1:
            keep_seasons = unique_seasons[-2:]  # last two seasons
        else:
            keep_seasons = unique_seasons
        features_df = features_df[features_df['season'].isin(keep_seasons)].copy()
        print(f"Including seasons: {keep_seasons}")

    # Confirm target column survived season filtering
    print(f"Columns after season filter: {features_df.columns.tolist()}")

    features_df = features_df.set_index('gameDate')
    window_sizes = [3, 5, 10]
    stat_cols = ['pts', 'reb', 'ast', 'stl', 'blk', 'mins']

    # No shift needed: data up to (but not including) TARGET_DATE
    print(f'\nCalculating rolling averages (windows: {window_sizes})...')
    for stat in stat_cols:
        if stat in features_df.columns:
            for window in window_sizes:
                col_name = f'{stat}_ma{window}'
                features_df[col_name] = features_df[stat].rolling(
                    window=window,
                    min_periods=1
                ).mean()
            for window in window_sizes:
                col_name = f'{stat}_std{window}'
                features_df[col_name] = features_df[stat].rolling(
                    window=window,
                    min_periods=1
                ).std()

    print('Calculating min/max features...')
    for stat in stat_cols:
        if stat in features_df.columns:
            for window in window_sizes:
                min_col = f'{stat}_min{window}'
                max_col = f'{stat}_max{window}'
                features_df[min_col] = features_df[stat].rolling(
                    window=window,
                    min_periods=1
                ).min()
                features_df[max_col] = features_df[stat].rolling(
                    window=window,
                    min_periods=1
                ).max()

    # Calculate EWMA features
    ewma_span = 3
    print('Calculating EWMA features...')
    for stat in stat_cols:
        if stat in features_df.columns:
            col_name = f'{stat}_ewma{ewma_span}'
            features_df[col_name] = features_df[stat].ewm(
                span=ewma_span,
                adjust=False
            ).mean()

    # Always explicitly include the target column
    feature_cols = [col for col in features_df.columns if any([
        '_ma' in col,
        '_std' in col,
        '_min' in col,
        '_max' in col,
        '_ewma' in col,
    ])] + ['target']

    print(f'\nTotal features created: {len(feature_cols)}')
    print(f'\nFeature categories:')
    print(f'  Averages: {len([c for c in feature_cols if "_avg_" in c])}')
    print(f'  Std devs: {len([c for c in feature_cols if "_std_" in c])}')
    print(f'  Min/Max: {len([c for c in feature_cols if "_min_" in c or "_max_" in c])}')
    print(f'  EWMA: {len([c for c in feature_cols if "_ewma" in c])}')
    print(f'  Target: {len([c for c in feature_cols if c=="target"])}')
    print(f'  Other: {len([c for c in feature_cols if c == "games_played"])}')

    # Print if target column is in DataFrame just before leaving this section
    print("'target' in features_df columns?", 'target' in features_df.columns)

Building features from historical data...
Restricting to only current and last season data...
Using .shift(1) to ensure no current-game leakage
Columns at target creation: ['playerID', 'longName', 'team', 'teamAbv', 'teamID', 'gameID', 'gameDate', 'pos', 'mins', 'pts', 'reb', 'ast', 'stl', 'blk', 'TOV', 'PF', 'fga', 'fgm', 'fgp', 'fta', 'ftm', 'ftp', 'tptfga', 'tptfgm', 'tptfgp', 'OffReb', 'DefReb', 'plusMinus', 'usage', 'tech', 'fpts', 'season', 'pts_season_avg', 'pts_season_std', 'reb_season_avg', 'reb_season_std', 'ast_season_avg', 'ast_season_std', 'stl_season_avg', 'stl_season_std', 'blk_season_avg', 'blk_season_std', 'TOV_season_avg', 'TOV_season_std', 'mins_season_avg', 'mins_season_std', 'fpts_season_avg', 'fpts_season_std', 'target']
Including seasons: [np.int64(2024), np.int64(2025)]
Columns after season filter: ['playerID', 'longName', 'team', 'teamAbv', 'teamID', 'gameID', 'gameDate', 'pos', 'mins', 'pts', 'reb', 'ast', 'stl', 'blk', 'TOV', 'PF', 'fga', 'fgm', 'fgp', 'fta',

In [69]:
display(features_df.head())

display(features_df.tail())

,playerID,longName,team,teamAbv,teamID,gameID,pos,mins,pts,reb,ast,stl,blk,TOV,PF,fga,fgm,fgp,fta,ftm,ftp,tptfga,tptfgm,tptfgp,OffReb,DefReb,plusMinus,usage,tech,fpts,season,pts_season_avg,pts_season_std,reb_season_avg,reb_season_std,ast_season_avg,ast_season_std,stl_season_avg,stl_season_std,blk_season_avg,blk_season_std,TOV_season_avg,TOV_season_std,mins_season_avg,mins_season_std,fpts_season_avg,fpts_season_std,target,pts_ma3,pts_ma5,pts_ma10,pts_std3,pts_std5,pts_std10,reb_ma3,reb_ma5,reb_ma10,reb_std3,reb_std5,reb_std10,ast_ma3,ast_ma5,ast_ma10,ast_std3,ast_std5,ast_std10,stl_ma3,stl_ma5,stl_ma10,stl_std3,stl_std5,stl_std10,blk_ma3,blk_ma5,blk_ma10,blk_std3,blk_std5,blk_std10,mins_ma3,mins_ma5,mins_ma10,mins_std3,mins_std5,mins_std10,pts_min3,pts_max3,pts_min5,pts_max5,pts_min10,pts_max10,reb_min3,reb_max3,reb_min5,reb_max5,reb_min10,reb_max10,ast_min3,ast_max3,ast_min5,ast_max5,ast_min10,ast_max10,stl_min3,stl_max3,stl_min5,stl_max5,stl_min10,stl_max10,blk_min3,blk_max3,blk_min5,blk_max5,blk_min10,blk_max10,mins_min3,mins_max3,mins_min5,mins_max5,mins_min10,mins_max10,pts_ewma3,reb_ewma3,ast_ewma3,stl_ewma3,blk_ewma3,mins_ewma3
gameDate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-10-24,28908111729,Nikola Jokic,DEN,DEN,8,20231024_LAL@DEN,,36,29,13,11,1,1,2,2,22,12,54.5,4,2,50.0,5,3,60.0,3,10,+15,31.72,0,67.75,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.5,29.000000,29.000000,29.000000,NaN,NaN,NaN,13.000000,13.00,13.00,NaN,NaN,NaN,11.000000,11.000000,11.000000,NaN,NaN,NaN,1.000000,1.000000,1.000000,NaN,NaN,NaN,1.000000,1.000000,1.000000,NaN,NaN,NaN,36.000000,36.000000,36.000000,NaN,NaN,NaN,29.0,29.0,29.0,29.0,29.0,29.0,13.0,13.0,13.0,13.0,13.0,13.0,11.0,11.0,11.0,11.0,11.0,11.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,36.0,36.0,36.0,36.0,36.0,36.0,29.0000,13.0000,11.0,1.000,1.000,36.0000
2023-10-27,28908111729,Nikola Jokic,DEN,DEN,8,20231027_DEN@MEM,,35,22,12,7,1,1,9,1,14,8,57.1,6,5,83.3,4,1,25.0,3,9,+6,30.6,0,48.50,2024,29.000000,NaN,13.00,NaN,11.000000,NaN,1.000000,NaN,1.000000,NaN,2.0,NaN,36.000000,NaN,67.7500,NaN,52.5,25.500000,25.500000,25.500000,4.949747,4.949747,4.949747,12.500000,12.50,12.50,0.707107,0.707107,0.707107,9.000000,9.000000,9.000000,2.828427,2.828427,2.828427,1.000000,1.000000,1.000000,0.00000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,35.500000,35.500000,35.500000,0.707107,0.707107,0.707107,22.0,29.0,22.0,29.0,22.0,29.0,12.0,13.0,12.0,13.0,12.0,13.0,7.0,11.0,7.0,11.0,7.0,11.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,35.0,36.0,35.0,36.0,35.0,36.0,25.5000,12.5000,9.0,1.000,1.000,35.5000
2023-10-29,28908111729,Nikola Jokic,DEN,DEN,8,20231029_DEN@OKC,,30,28,14,5,0,0,4,3,16,12,75.0,3,3,100.0,2,1,50.0,2,12,+21,31.96,0,52.50,2024,25.500000,4.949747,12.50,0.707107,9.000000,2.828427,1.000000,0.00000,1.000000,0.000000,5.5,4.949747,35.500000,0.707107,58.1250,13.611806,62.5,26.333333,26.333333,26.333333,3.785939,3.785939,3.785939,13.000000,13.00,13.00,1.000000,1.000000,1.000000,7.666667,7.666667,7.666667,3.055050,3.055050,3.055050,0.666667,0.666667,0.666667,0.57735,0.577350,0.577350,0.666667,0.666667,0.666667,0.577350,0.577350,0.577350,33.666667,33.666667,33.666667,3.214550,3.214550,3.214550,22.0,29.0,22.0,29.0,22.0,29.0,12.0,14.0,12.0,14.0,12.0,14.0,5.0,11.0,5.0,11.0,5.0,11.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,30.0,36.0,30.0,36.0,30.0,36.0,26.7500,13.2500,7.0,0.500,0.500,32.7500
2023-10-30,28908111729,Nikola Jokic,DEN,DEN,8,20231030_UTA@DEN,,35,27,10,11,0,2,1,3,16,12,75.0,5,2,40.0,3,1,33.3,1,9,+13,25.2,0,62.50,2024,26.333333,3.785939,13.00,1.000000,7.666667,3.055050,0.666667,0.57735,0.666667,0.577350,5.0,3.605551,33.666667,3.214550,56.2500,10.158125,41.0,25.666667,26.500000,26.500000,3.214550,3.109126,3.109126,12.000000,12.25,12.25,2.000000,1.707825,1.707825,7.666667,8.500000,8.500000,3.055050,3.000000,3.000000,0.333333,0.500000,0.500000,0.57735,

,playerID,longName,team,teamAbv,teamID,gameID,pos,mins,pts,reb,ast,stl,blk,TOV,PF,fga,fgm,fgp,fta,ftm,ftp,tptfga,tptfgm,tptfgp,OffReb,DefReb,plusMinus,usage,tech,fpts,season,pts_season_avg,pts_season_std,reb_season_avg,reb_season_std,ast_season_avg,ast_season_std,stl_season_avg,stl_season_std,blk_season_avg,blk_season_std,TOV_season_avg,TOV_season_std,mins_season_avg,mins_season_std,fpts_season_avg,fpts_season_std,target,pts_ma3,pts_ma5,pts_ma10,pts_std3,pts_std5,pts_std10,reb_ma3,reb_ma5,reb_ma10,reb_std3,reb_std5,reb_std10,ast_ma3,ast_ma5,ast_ma10,ast_std3,ast_std5,ast_std10,stl_ma3,stl_ma5,stl_ma10,stl_std3,stl_std5,stl_std10,blk_ma3,blk_ma5,blk_ma10,blk_std3,blk_std5,blk_std10,mins_ma3,mins_ma5,mins_ma10,mins_std3,mins_std5,mins_std10,pts_min3,pts_max3,pts_min5,pts_max5,pts_min10,pts_max10,reb_min3,reb_max3,reb_min5,reb_max5,reb_min10,reb_max10,ast_min3,ast_max3,ast_min5,ast_max5,ast_min10,ast_max10,stl_min3,stl_max3,stl_min5,stl_max5,stl_min10,stl_max10,blk_min3,blk_max3,blk_min5,blk_max5,blk_min10,blk_max10,mins_min3,mins_max3,mins_min5,mins_max5,mins_min10,mins_max10,pts_ewma3,reb_ewma3,ast_ewma3,stl_ewma3,blk_ewma3,mins_ewma3
gameDate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-02-01,28908111729,Nikola Jokic,DEN,DEN,8,20250201_DEN@CHA,,37,28,13,17,4,1,4,2,17,9,52.9,9,8,88.9,7,2,28.6,1,12,-2,30.04,0,80.75,2025,29.642857,9.336060,12.833333,4.751979,10.142857,3.719241,1.809524,1.234427,0.642857,0.790845,3.166667,1.793167,36.404762,4.030733,66.220238,14.978766,68.00,24.333333,25.2,24.2,6.350853,6.534524,7.598245,9.333333,8.6,11.8,3.511885,4.159327,5.116422,12.000000,12.2,11.8,5.567764,4.086563,3.457681,2.666667,1.8,2.0,1.154701,1.483240,1.333333,0.666667,0.6,0.9,0.57735,0.547723,0.994429,35.666667,35.8,33.6,2.309401,3.271085,3.835507,17.0,28.0,17.0,33.0,10.0,35.0,6.0,13.0,3.0,13.0,3.0,22.0,6.0,17.0,6.0,17.0,6.0,17.0,2.0,4.0,0.0,4.0,0.0,4.0,0.0,1.0,0.0,1.0,0.0,3.0,33.0,37.0,32.0,40.0,29.0,40.0,26.732760,10.895793,14.141442,2.877885,0.738557,36.431436
2025-02-03,28908111729,Nikola Jokic,DEN,DEN,8,20250203_NO@DEN,,36,27,14,10,1,2,1,2,13,9,69.2,10,8,80.0,3,1,33.3,1,13,+11,22.42,0,68.00,2025,29.604651,9.227649,12.837209,4.695135,10.302326,3.820589,1.860465,1.264561,0.651163,0.783269,3.186047,1.776243,36.418605,3.983493,66.558140,14.964326,68.00,27.666667,26.6,25.9,0.577350,5.856620,5.743595,12.000000,10.8,11.8,2.645751,3.271085,5.116422,13.333333,12.0,11.8,3.511885,4.183300,3.457681,2.333333,2.0,1.9,1.527525,1.224745,1.370320,1.000000,1.0,1.1,1.00000,0.707107,0.994429,36.666667,36.6,34.3,0.577350,2.509980,3.529243,27.0,28.0,17.0,33.0,17.0,35.0,9.0,14.0,6.0,14.0,3.0,22.0,10.0,17.0,6.0,17.0,6.0,17.0,1.0,4.0,1.0,4.0,0.0,4.0,0.0,2.0,0.0,2.0,0.0,3.0,36.0,37.0,33.0,40.0,30.0,40.0,26.866380,12.447897,12.070721,1.938943,1.369278,36.215718
2025-02-05,28908111729,Nikola Jokic,DEN,DEN,8,20250205_NO@DEN,,36,38,8,10,1,1,1,3,28,15,53.6,3,3,100.0,13,5,38.5,2,6,+24,34.97,0,68.00,2025,29.545455,9.128169,12.863636,4.643530,10.295455,3.776177,1.840909,1.256484,0.681818,0.800370,3.136364,1.786134,36.409091,3.937407,66.590909,14.790896,64.00,31.000000,27.6,27.3,6.082763,7.436397,6.832114,11.666667,10.0,11.4,3.214550,3.391165,5.253570,12.333333,11.2,11.8,4.041452,4.086563,3.457681,2.000000,2.0,1.9,1.732051,1.224745,1.370320,1.333333,1.0,1.2,0.57735,0.707107,0.918937,36.333333,35.8,34.8,0.577350,1.643168,3.359894,27.0,38.0,17.0,38.0,17.0,38.0,8.0,14.0,6.0,14.0,3.0,22.0,10.0,17.0,6.0,17.0,6.0,17.0,1.0,4.0,1.0,4.0,0.0,4.0,1.0,2.0,0.0,2.0,0.0,3.0,36.0,37.0,33.0,37.0,30.0,40.0,32.433190,10.223948,11.035360,1.469471,1.184639,36.107859
2025-02-06,28908111729,Nikola Jokic,DEN,DEN,8,20250206_ORL@DEN,,31,28,10,12,2,0,3,2,16,11,68.8,6,4,66.7,4,2,50.0,0,10,+22,29.85,0,64.00,2025,29.733333,9.111431,12.755556,4.647363,10.288889,3.733279,1.822222,1.248433,0.688889,0.792643,3.088889,1.794211,36.400000,3.892884,66.622222,14.623361,53.75,31.000000,29.8,28.1,6.082763,4.604346,6.3324

## Basic Statistics Summary

## View Features for Most Recent Game

These are the features that would be used to predict TARGET_DATE performance.

In [61]:
if selected_player is not None and not player_data.empty:
    print(f'Features for {player_name} on {TARGET_DATE}:')
    print('(These would be fed to the model for prediction)')
    print('='*60)
    
    latest_features = features_df[feature_cols].iloc[-1]
    
    print(f'\nFantasy Points Features:')
    fpts_features = latest_features[[col for col in latest_features.index if col.startswith('fpts')]]
    for feature, value in fpts_features.items():
        print(f'  {feature:30s} {value:8.2f}')
    
    print(f'\nPoints Features:')
    pts_features = latest_features[[col for col in latest_features.index if col.startswith('pts')]]
    for feature, value in pts_features.items():
        print(f'  {feature:30s} {value:8.2f}')
    
    print(f'\nMinutes Features:')
    mins_features = latest_features[[col for col in latest_features.index if col.startswith('mins')]]
    for feature, value in mins_features.items():
        print(f'  {feature:30s} {value:8.2f}')

    print(f'\nAll {len(latest_features)} features as DataFrame:')
    features_display = pd.DataFrame({
        'Feature': latest_features.index,
        'Value': latest_features.values
    }).sort_values('Feature')
    
    display(features_display)

Features for Nikola Jokic on 20250210:
(These would be fed to the model for prediction)

Fantasy Points Features:

Points Features:
  pts_avg_3                         30.67
  pts_avg_5                         29.40
  pts_avg_10                        28.00
  pts_std_3                          6.43
  pts_std_5                          4.88
  pts_std_10                         6.36
  pts_min_3                         26.00
  pts_max_3                         38.00
  pts_min_5                         26.00
  pts_max_5                         38.00
  pts_min_10                        17.00
  pts_max_10                        38.00
  pts_ewma                          29.57

Minutes Features:
  mins_avg_3                        32.00
  mins_avg_5                        33.80
  mins_avg_10                       34.80
  mins_std_3                         3.61
  mins_std_5                         3.56
  mins_std_10                        3.39
  mins_min_3                        29.00
  mins_ma

,Feature,Value
14,ast_avg_10,11.900000
12,ast_avg_3,10.333333
13,ast_avg_5,11.600000
74,ast_ewma,11.612222
53,ast_max_10,17.000000
49,ast_max_3,12.000000
51,ast_max_5,17.000000
52,ast_min_10,6.000000
48,ast_min_3,9.000000
50,ast_min_5,9.000000
